<a href="https://colab.research.google.com/github/ewotawa/secure_private_ai/blob/master/Section_2_Federated_Learning_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Federated Learning Final Project

## Overview
* See  <a href="https://classroom.udacity.com/nanodegrees/nd185/parts/3fe1bb10-68d7-4d84-9c99-9539dedffad5/modules/28d685f0-0cb1-4f94-a8ea-2e16614ab421/lessons/c8fe481d-81ea-41be-8206-06d2deeb8575/concepts/a5fb4b4c-e38a-48de-b2a7-4e853c62acbe">video</a> for additional details. 
* Do Federated Learning where the central server is not trusted with the raw gradients.  
* In the final project notebook, you'll receive a dataset.  
* Train on the dataset using Federated Learning.  
* The gradients should not come up to the server in raw form.  
* Instead, use the new .move() command to move all of the gradients to one of the workers, sum them up there, and then bring that batch up to the central server and then bring that batch up 
* Idea: the central server never actually sees the raw gradient for any person.  
* We'll look at secure aggregation in course 3.  
* For now, do a larger-scale Federated Learning case where you handle the gradients in a special way.

## References
*  <a href = "https://blog.openmined.org/upgrade-to-federated-learning-in-10-lines/">DEEP LEARNING -> FEDERATED LEARNING IN 10 LINES OF PYTORCH + PYSYFT</a>
* <a href ="https://github.com/udacity/private-ai/pull/10">added data for Federated Learning project</a>
* <a href="https://github.com/OpenMined/PySyft/blob/master/examples/tutorials/Part%206%20-%20Federated%20Learning%20on%20MNIST%20using%20a%20CNN.ipynb">Part 6 - Federated Learning on MNIST using a CNN.ipynb</a>

### Install libraries and dependencies

In [1]:
# PyTorch
# Install Dependencies
# https://colab.research.google.com/drive/1gJAAN3UI9005ecVmxPun5ZLCGu4YBtLo#scrollTo=XiHYo3hEhwdk

# !pip3 install torch torchvision

!pip install --upgrade --force-reinstall torch
!pip install --upgrade --force-reinstall torchvision



  Using cached https://files.pythonhosted.org/packages/69/60/f685fb2cfb3088736bafbc9bdbb455327bdc8906b606da9c9a81bae1c81e/torch-1.1.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/87/2d/e4656149cbadd3a8a0369fcd1a9c7d61cc7b87b3903b85389c70c989a696/numpy-1.16.4-cp36-cp36m-manylinux1_x86_64.whl
ERROR: keras-vis 0.4.1 requires scikit-image, which is not installed.
ERROR: imgaug 0.2.9 requires scikit-image>=0.11.0, which is not installed.
ERROR: google-colab 1.0.0 has requirement requests~=2.21.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.9.1 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4
  Found existing installation: torch 1.1.0
    Uninstallin

  Using cached https://files.pythonhosted.org/packages/2e/45/0f2f3062c92d9cf1d5d7eabd3cae88cea9affbd2b17fb1c043627838cb0a/torchvision-0.3.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/87/2d/e4656149cbadd3a8a0369fcd1a9c7d61cc7b87b3903b85389c70c989a696/numpy-1.16.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/69/60/f685fb2cfb3088736bafbc9bdbb455327bdc8906b606da9c9a81bae1c81e/torch-1.1.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/14/41/db6dec65ddbc176a59b89485e8cc136a433ed9c6397b6bfe2cd38412051e/Pillow-6.1.0-cp36-cp36m-manylinux1_x86_64.whl
ERROR: keras-vis 0.4.1 requires scikit-image, which is not installed.
ERROR: imgaug 0.2.9 requires scikit-image>=0.11.0, which is not installed.
ERROR: kaggle 1.5.4 has r

In [2]:
# PySyft
# Install prerequisite for install

# ERROR: syft 0.1.19a1 has requirement msgpack>=0.6.1, but you'll have msgpack 0.5.6 which is incompatible.

!pip install --upgrade --force-reinstall msgpack

  Using cached https://files.pythonhosted.org/packages/92/7e/ae9e91c1bb8d846efafd1f353476e3fd7309778b582d2fb4cea4cc15b9a2/msgpack-0.6.1-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: msgpack 0.6.1
    Uninstalling msgpack-0.6.1:
      Successfully uninstalled msgpack-0.6.1


In [3]:
# PySyft
# Install dependencies

# https://github.com/OpenMined/PySyft
# https://colab.research.google.com/drive/14tNU98OKPsP55Y3IgFtXPfd4frqbkrxK#scrollTo=qpX6fqECKGyk

!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

From https://github.com/openmined/PySyft
 * branch              HEAD       -> FETCH_HEAD
Already up to date.
zip_safe flag not set; analyzing archive contents...
  Using cached https://files.pythonhosted.org/packages/0a/c6/96bbb3525a63ebc53ea700cc7d37ab9045542d33b4d262d0f0408ad9bbf2/lz4-2.1.10-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: lz4 2.1.10
    Uninstalling lz4-2.1.10:
      Successfully uninstalled lz4-2.1.10
  Using cached https://files.pythonhosted.org/packages/bf/45/142141aa47e01a5779f0fa5a53b81f8379ce8f2b1cd13df7d2f1d751ae42/greenlet-0.4.15-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f2/ca/5b5962361ed832847b6b2f9a2d0452c8c2f29a93baef850bb8ad067c7bf9/gevent-1.4.0-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: greenlet 0.4.15
    Uninstalling greenlet-0.4.15:
      Successfully uninstalled greenlet-0.4.15
  Found existing installation: gevent 1.4.0
    Uninstalling gevent-1.4.0:
      Successfull

In [4]:
# PySyft
# make sure nothing's missing.

!pip install syft
!pip install numpy

In [5]:
# Import libraries
# https://colab.research.google.com/drive/14tNU98OKPsP55Y3IgFtXPfd4frqbkrxK#scrollTo=ZgomH7s4R5cT

#from __future__ import print_function
#import argparse
#import numpy as np
#import pandas as pd
#import torch.utils
#import torch.utils.data

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

torch.__version__

'1.1.0'

In [6]:
# Import PySyft library

import syft

syft.__version__

W0716 20:30:59.425365 140409394038656 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0716 20:30:59.457098 140409394038656 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



'0.1.21a1'

Walk through a Federated Learning exercise to train the MNIST dataset (see references above). Modify the exercise to meet the requirements of the final project. 

In [68]:
# Define the workers (here, use virtual workers). Need several to distribute the federated learning work, one aggregator

hook = syft.TorchHook(torch)

vw00 = syft.VirtualWorker(hook, id="vw00")
vw01 = syft.VirtualWorker(hook, id="vw01")
vw02 = syft.VirtualWorker(hook, id="vw02")
vw03 = syft.VirtualWorker(hook, id="vw03")
vw04 = syft.VirtualWorker(hook, id="vw04")
aggr = syft.VirtualWorker(hook, id="aggr")

W0716 21:43:20.510649 140409394038656 hook.py:98] Torch was already hooked... skipping hooking process


In [69]:
# Load the data into trainloader and testloader, send it to the workers. 

trsf = transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])

federated_trainloader = syft.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=trsf)
    .federate((vw00, vw01, vw02, vw03, vw04)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

testloader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=trsf),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

W0716 21:43:42.632259 140409394038656 dataloader.py:197] The following options are not supported: num_workers: 1, pin_memory: True


In [0]:
# Define the classification model to be used on each of the private loaders.
# Theoretically could be six different models. Training the same model on each dataset here for expediency.
# See Prior work. Exercise Part 5 of CNNs.

## TODO: Define your model with dropout added

from torch import nn, optim
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
        # add dropout module with 0.2 drop probability
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        # update to include dropout
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        
        # no dropout on output layer
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

In [0]:
#for batch_idx, (data, target) in enumerate(federated_trainloader):

class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

torch.manual_seed(args.seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [0]:
# Function to train the model on each of the data loaders for a specified number of epochs, number of steps, and optimizer learning rate.

def TrainModel(args, trainloader, testloader, lr, epochs, steps):
  
  model = Classifier()
  
  # move the model to GPU, if available
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  criterion = nn.NLLLoss()
  optimizer = optim.Adam(model.parameters(), lr)

  # epochs = 30
  # steps = 0
  
  train_losses, test_losses, test_accuracy = [], [], []
  for e in range(epochs):
      running_loss = 0
      #for images, labels in trainloader:
      for batch_idx, (data, target) in enumerate(federated_trainloader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        # data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        
        log_ps = model(data)
        loss = criterion(log_ps, target)
        loss.backward()
        optimizer.step()

        loss = loss.get()
        running_loss += loss
          
        #output = model(data)
        #loss = F.nll_loss(output, target)
        #loss.backward()
        #optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            # loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_trainloader) * args.batch_size,
                100. * batch_idx / len(federated_trainloader), loss.item()))

      else:
          ## TODO: Implement the validation pass and print out the validation accuracy
          # turn off gradients
          test_loss = 0
          accuracy = 0

          # turn off gradients
          with torch.no_grad():

            # set model to evaluation mode
            model.eval()

            # run validation pass
            for images, labels in testloader:

              optimizer.zero_grad()

              log_ps = model(images)
              test_loss = test_loss + criterion(log_ps, labels)

              # Get the class probabilities
              ps = torch.exp(log_ps)

              # Use top-k to identify most likely classes
              top_p, top_class = ps.topk(1, dim=1)

              # compare to true labels
              equals = top_class == labels.view(*top_class.shape)
              accuracy = accuracy + torch.mean(equals.type(torch.FloatTensor))

          # set model back to training mode
          model.train()

          train_losses.append(running_loss/len(trainloader))
          test_losses.append(test_loss/len(testloader))
          test_accuracy.append(accuracy/len(testloader))

          # print(f'Accuracy: {accuracy.item()*100}%')

          #print("Epoch: {}/{}.. ".format(e+1, epochs),
          #  "Training loss: {:.3f}.. ".format(running_loss/len(trainloader)),
          #  "Testing loss: {:.3f}.. ".format(test_loss/len(testloader)),
          #  "Test Accuracy: {:.3f}.. ".format(accuracy/len(testloader))
          #  )
  
  plt.plot(train_losses, label='Training Loss')
  plt.plot(test_losses, label='Validation Loss')
  plt.plot(test_accuracy, label='Test Accuracy')
  plt.legend(frameon=False)

  return model

In [97]:
model_0 = TrainModel(args=args, trainloader=federated_trainloader, testloader=testloader, lr=args.lr, epochs=args.epochs, steps=0)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.303935
Train Epoch: 1 [1920/60032 (3%)]	Loss: 0.912373
Train Epoch: 1 [3840/60032 (6%)]	Loss: 0.648745
Train Epoch: 1 [5760/60032 (10%)]	Loss: 0.527316
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.287380
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.258494
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.768991


TypeError: ignored

In [57]:
# Launch the training

# model = cnn_model.to(device)

optimizer = optim.SGD(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
  train(args, model, device, federated_trainloader, optimizer, epoch)
  test(args, model, device, testloader)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.319468
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.295157
Train Epoch: 1 [1280/60032 (2%)]	Loss: 2.300367
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.286685
Train Epoch: 1 [2560/60032 (4%)]	Loss: 2.307686
Train Epoch: 1 [3200/60032 (5%)]	Loss: 2.287722
Train Epoch: 1 [3840/60032 (6%)]	Loss: 2.291087
Train Epoch: 1 [4480/60032 (7%)]	Loss: 2.282883
Train Epoch: 1 [5120/60032 (9%)]	Loss: 2.253607
Train Epoch: 1 [5760/60032 (10%)]	Loss: 2.259088
Train Epoch: 1 [6400/60032 (11%)]	Loss: 2.271262
Train Epoch: 1 [7040/60032 (12%)]	Loss: 2.227728
Train Epoch: 1 [7680/60032 (13%)]	Loss: 2.233532
Train Epoch: 1 [8320/60032 (14%)]	Loss: 2.218377
Train Epoch: 1 [8960/60032 (15%)]	Loss: 2.209551
Train Epoch: 1 [9600/60032 (16%)]	Loss: 2.210539
Train Epoch: 1 [10240/60032 (17%)]	Loss: 2.176412
Train Epoch: 1 [10880/60032 (18%)]	Loss: 2.143712
Train Epoch: 1 [11520/60032 (19%)]	Loss: 2.173175
Train Epoch: 1 [12160/60032 (20%)]	Loss: 2.164583
Train Epoch: 1 [12800/60032 (

Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 234, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/lib/python3.6/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
_pickle.PicklingError: Can't pickle <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>: attribute lookup Tensor on syft.frameworks.torch.tensors.interpreters.native failed
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 234, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/lib/python3.6/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
_pickle.PicklingError: Can't pickle <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>: attribute lookup Tensor on syft.frameworks.torch.tensors.interpreters.native failed


KeyboardInterrupt: ignored

In [0]:
print("Model: ", model)

In [0]:
if(args.save_model):
  torch.save(model.state_dict(), "mnist_cnn.pt")